In [1]:
#CELDA 1: Configuraciones básicas

import re
import pandas as pd
from pathlib import Path
import numpy as np

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

%run 00_setup.ipynb

#RUTAS
from src.config.rutas import (
    OUTPUT_DIR,
    RUTA_DATASET,
    RANDOM_STATE,
    TEST_SIZE
)

In [2]:
#CELDA 2: Carga del dataset

dt = pd.read_csv(RUTA_DATASET)

print("Num. de columnas y filas:")
print(dt.shape)
dt.head()

Num. de columnas y filas:
(16127, 5)


,texto,label,vector,fuente,id
0,Moby Pub Quiz.Win a £100 High Street prize if ...,1,sms,mendeley,0
1,https://www.documenters.org,0,url,phiusiil,1
2,scan system adware wrongful 6130320948821283 c...,1,email,kaggle_email,2
3,mit student thesis writeups hi vince krishna t...,0,email,kaggle_email,3
4,https://swn9klhal.web.app/,1,url,phiusiil,4


In [3]:
#CELDA 3: Función de limpieza de texto

def limpiar_texto(t: str) -> str:

    #Se realiza una limpieza ligera y segura para los tres tipos de vecrores principales preservando la semantica
    if not isinstance(t, str):
        t = str(t)
    
    t = t.lower()                           #Convertir a minusculas
    t = re.sub(r"\s", " ", t)               #Elimina saltos de linea
    t = re.sub(r"[^\x20-\x7E]", "", t)      #Elimina carcteres no imprimibles
    t = t.strip()                           #Compacta los espacios

    return t

In [4]:
#CELDA 5: Aplicar la funcion de limpieza

dt["texto_preprocesado"] = dt["texto"].apply(limpiar_texto)

print("Conversion:")
display(dt[["texto", "texto_preprocesado"]].head())

#Comprobacion post-limpieza

print("\nTextos vacios post-limpieza:")
display((dt["texto_preprocesado"].str.len() == 0).sum())

print("\nEjemplos aleatorios post-limpieza:")
display(dt.sample(5)[["texto", "texto_preprocesado"]])

print("\nURLs post-limpieza:")
display(dt[dt["vector"] == "url"].sample(5)[["texto", "texto_preprocesado"]])

Conversion:


,texto,texto_preprocesado
0,Moby Pub Quiz.Win a £100 High Street prize if ...,moby pub quiz.win a 100 high street prize if u...
1,https://www.documenters.org,https://www.documenters.org
2,scan system adware wrongful 6130320948821283 c...,scan system adware wrongful 6130320948821283 c...
3,mit student thesis writeups hi vince krishna t...,mit student thesis writeups hi vince krishna t...
4,https://swn9klhal.web.app/,https://swn9klhal.web.app/



Textos vacios post-limpieza:


np.int64(1)


Ejemplos aleatorios post-limpieza:


,texto,texto_preprocesado
9986,fred inklaar opslaginklaarnet op 12092002 0035...,fred inklaar opslaginklaarnet op 12092002 0035...
15323,http://www.211207bei.top,http://www.211207bei.top
10594,https://my-site-101353-100233.weeblysite.com/,https://my-site-101353-100233.weeblysite.com/
12503,You are entitled to update to the latest colou...,you are entitled to update to the latest colou...
5381,offsite meeting great divide lodge invited gue...,offsite meeting great divide lodge invited gue...



URLs post-limpieza:


,texto,texto_preprocesado
10316,http://www.xn--enyakn-t9a.com,http://www.xn--enyakn-t9a.com
12120,https://www.appysmarts.com,https://www.appysmarts.com
9580,https://aol-101047.weeblysite.com/,https://aol-101047.weeblysite.com/
1949,https://www.torresette.news,https://www.torresette.news
4055,http://www.telegramself.ml,http://www.telegramself.ml


In [5]:
#CELDA 6: Eliminar textos vacios tras el preprocesamiento

num_vacios = (dt["texto_preprocesado"].str.len() == 0).sum()
print("Textos vacios: ", num_vacios)

dt = dt[dt["texto_preprocesado"].str.len() > 0].reset_index(drop=True)

print("Dataset final tras eliminar vacios:", dt.shape)

Textos vacios:  1
Dataset final tras eliminar vacios: (16126, 6)


In [6]:
#CELDA 7: Dividir en train/test (estratificado)

#Se crea la columna estrato combinando vector y label
dt["estrato"] = dt["vector"].astype(str) + "_" + dt["label"].astype(str)

print("Division de estratos:")
print(dt["estrato"].value_counts())

#SE configura el StratifiedShuffleSplit
sss = StratifiedShuffleSplit(
    n_splits=1,                     #Unica particion en train/test
    test_size=TEST_SIZE,            #Divide la particion del test en un 20 %
    random_state=RANDOM_STATE       #Se pasa la semilla 42 para que sea reproducible
)

#Por cada indice de las filas en cada conjunto se selecciona por posicion nnumerica y las asigna al subconjunto definido
for train_idx, test_idx in sss.split(dt, dt["estrato"]):
    dt_train = dt.iloc[train_idx].reset_index(drop=True)
    dt_test = dt.iloc[test_idx].reset_index(drop=True)

print("\nTamaños:")
print("Train:", dt_train.shape)
print("Test:", dt_test.shape)

#Ajustamos opciones de visulalizacion para que deje de mostrar los ultimos resultado obtenidos
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("\nDistribucion train por vector y label:")
print(dt_train.groupby(["vector", "label"]).size())

print("\nDistribucion test por vector y label:")
print(dt_test.groupby(["vector", "label"]).size())

Division de estratos:
estrato
url_0      3000
email_0    3000
sms_0      3000
url_1      3000
email_1    2999
sms_1      1127
Name: count, dtype: int64

Tamaños:
Train: (12900, 7)
Test: (3226, 7)

Distribucion train por vector y label:
vector  label
email   0        2400
        1        2399
sms     0        2400
        1         901
url     0        2400
        1        2400
dtype: int64

Distribucion test por vector y label:
vector  label
email   0        600
        1        600
sms     0        600
        1        226
url     0        600
        1        600
dtype: int64


In [7]:
#CELDA 8: Ajuste del vectorizador TF-IDF

x_train_text = dt_train["texto_preprocesado"].tolist()
x_test_text = dt_test["texto_preprocesado"].tolist()

#Se crea el vectorizador con los parametros
tfid = TfidfVectorizer(
    ngram_range=(1, 2),         #Se usan unigramas y bigramas
    min_df=5,                   #Se elimina palabras que aparecen menos de 5 veces, ya que menos cantidad de 5 reudce el ruido
    max_df=0.9,                 #Se ignora cuando aparecen mas del 90%
    max_features=50000,         #Se evita generar matrices gigantes
)

#Ajusta solo el conjunto de entrenamiento
tfid.fit(x_train_text)

#Se obtiene el tamaño del vocaubulario
voc_size = len(tfid.get_feature_names_out())
print("Tamaño del vocabulario TF-IDF:", voc_size)

Tamaño del vocabulario TF-IDF: 22675


In [8]:
#CELDA 9: Transformacion de texto a matrices TF-IDF

#Matriz de caracterisitcas al convertir texto en vectores numericos segun TF-IDF.
x_train = tfid.transform(x_train_text)
x_test = tfid.transform(x_test_text)

#Vectores de etiquetas obtenidas a traves de arrays de NumPy (formato requerido para los modelos)
y_train = dt_train["label"].to_numpy()
y_test = dt_test["label"].to_numpy()

print("Forma x_train", x_train.shape)
print("Forma x_test", x_test.shape)
print("Forma y_train", y_train.shape)
print("Forma y_test", y_test.shape)



Forma x_train (12900, 22675)
Forma x_test (3226, 22675)
Forma y_train (12900,)
Forma y_test (3226,)


In [9]:
#CELDA 10: Comprobacion de la dispersion de la matriz

#Calcula la dispersion, es decir, el porcentaje de ceros de un matriz dispersa.
disp_m = 1.0 - (x_train.count_nonzero() / float(x_train.shape[0] * x_train.shape[1]))   #Valores que no son cero / total de celdas
print(f"Dispersion aproximada de x_train: {disp_m:.4f}")    #Un alto porcentaje puede evitar ruido y reducir memoria

Dispersion aproximada de x_train: 0.9980


In [10]:
#CELDA 11: Guardar vectorizacion e informacion de las divisiones train/test

VECTORIZADOR_PATH = OUTPUT_DIR / "tfidf_vectorizador_v1.joblib"
TRAIN_PATH = OUTPUT_DIR / "train_v1.csv"
TEST_PATH = OUTPUT_DIR / "test_v1.csv"

#Se guarda el vectorizador TF-IDF
joblib.dump(tfid, VECTORIZADOR_PATH)

metadatos_train_test = ["id", "vector", "label"]
dt_train[metadatos_train_test].to_csv(TRAIN_PATH, index=False)
dt_test[metadatos_train_test].to_csv(TEST_PATH, index=False)

print("Vectorizador guardado en:", VECTORIZADOR_PATH)
print("Split de train guardado en:", TRAIN_PATH)
print("Split de test guardado en:", TEST_PATH)

Vectorizador guardado en: C:\TFG\models\tfidf_vectorizador_v1.joblib
Split de train guardado en: C:\TFG\models\train_v1.csv
Split de test guardado en: C:\TFG\models\test_v1.csv
